In [165]:
HCV_df=read.csv(file = '/Users/shreya/Downloads/hcvdat0.csv')

In [166]:
head(HCV_df)

,X,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106,12.1,69.0
2,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74,15.6,76.5
3,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86,33.2,79.3
4,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80,33.8,75.7
5,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76,29.9,68.7
6,6,0=Blood Donor,32,m,41.6,43.3,18.5,19.7,12.3,9.92,6.05,111,91.0,74.0


In [167]:
#Encode Y column to 0-3
HCV_df["Category"][HCV_df["Category"] == '0=Blood Donor'] <- 0
HCV_df["Category"][HCV_df["Category"] == '0s=suspect Blood Donor'] <- 0
HCV_df["Category"][HCV_df["Category"] == '1=Hepatitis'] <- 1
HCV_df["Category"][HCV_df["Category"] == '2=Fibrosis'] <- 2
HCV_df["Category"][HCV_df["Category"] == '3=Cirrhosis'] <- 3
#Encode Sex column to numeric: Male is 0, Female is 1
HCV_df["Sex"][HCV_df["Sex"] == 'm'] <- 0
HCV_df["Sex"][HCV_df["Sex"] == 'f'] <- 1
#Remove column of Patient ID
HCV_df <- subset (HCV_df, select = -X)

In [168]:
#Convert Y to a factor variable
HCV_df$Category=as.factor(HCV_df$Category)

In [169]:
#Distribution of classes in the datset
library(dplyr)
HCV_df %>%  group_by(Category) %>% summarise(my_counts = n())

Category,my_counts
<fct>,<int>
0,540
1,24
2,21
3,30


In [170]:
#Split the data into the training and test data with the same proportion classes as in the original data
library(caret)
set.seed(3456)
trainIndex <- createDataPartition(HCV_df$Category, p = 0.70,
                                  list = FALSE,
                                  times = 1)
Train <- HCV_df[ trainIndex,]
Test <- HCV_df[-trainIndex,]

In [171]:
#Data Check: Proportions are maintained within the splits
library(dplyr)
Train %>%  group_by(Category) %>% summarise(my_counts = n())

library(dplyr)
Test %>%  group_by(Category) %>% summarise(my_counts = n())

Category,my_counts
<fct>,<int>
0,378
1,17
2,15
3,21


Category,my_counts
<fct>,<int>
0,162
1,7
2,6
3,9


In [172]:
Train %>% group_by(Category) %>% summarise(Mean=mean(CHOL, na.rm=TRUE), Median=median(CHOL, na.rm=TRUE)) 

Category,Mean,Median
<fct>,<dbl>,<dbl>
0,5.495791,5.400
1,5.071765,5.370
2,4.340714,4.380
3,3.676500,3.655


In [173]:
which(is.na(Train$CHOL))

[1] 216 225 287 295 350 410 424

In [174]:
Train[216,]

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
,<fct>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
320,0,32,1,47.4,52.5,19.1,17.1,4.6,10.19,NA,63,23,72.2


In [175]:
#Impute values using column medians - TRAIN
library(dplyr)
Train=Train %>% group_by(Category) %>%   mutate(ALT = ifelse(is.na(ALT), 
                            median(ALT, na.rm = TRUE), 
                            ALT))

Train=Train %>% group_by(Category) %>%   mutate(PROT = ifelse(is.na(PROT), 
                            median(PROT, na.rm = TRUE), 
                            PROT))

Train=Train %>% group_by(Category) %>%   mutate(CHOL = ifelse(is.na(CHOL), 
                            median(CHOL, na.rm = TRUE), 
                            CHOL))

Train=Train %>% group_by(Category) %>%   mutate(ALB = ifelse(is.na(ALB), 
                            median(ALB, na.rm = TRUE), 
                            ALB))

Train=Train %>% group_by(Category) %>%   mutate(ALP = ifelse(is.na(ALP), 
                            median(ALT, na.rm = TRUE), 
                            ALP))

In [176]:
Train[216,]

Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
<fct>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,32,1,47.4,52.5,19.1,17.1,4.6,10.19,5.4,63,23,72.2


In [177]:
#Impute values using column medians - TEST
Test=Test %>% group_by(Category) %>%   mutate(ALT = ifelse(is.na(ALT), 
                            median(ALT, na.rm = TRUE), 
                            ALT))

Test=Test %>% group_by(Category) %>%   mutate(PROT = ifelse(is.na(PROT), 
                            median(PROT, na.rm = TRUE), 
                            PROT))

Test=Test %>% group_by(Category) %>%   mutate(CHOL = ifelse(is.na(CHOL), 
                            median(CHOL, na.rm = TRUE), 
                            CHOL))

Test=Test %>% group_by(Category) %>%   mutate(ALB = ifelse(is.na(ALB), 
                            median(ALB, na.rm = TRUE), 
                            ALB))

Test=Test %>% group_by(Category) %>%   mutate(ALP = ifelse(is.na(ALP), 
                            median(ALT, na.rm = TRUE), 
                            ALP))

In [178]:
#Data Check: no null rows remaining
Train[rowSums(is.na(Train)) > 0,]

Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
<fct>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [179]:
#Convert to train_x, train_y, test_x, test_y
train_labels=Train[["Category"]]
test_labels=Test[['Category']]
train_x <- subset( Train, select = -Category )
test_x <- subset( Test, select = -Category )

In [180]:
#store as matrix form
train_labels = as.matrix(train_labels)
test_labels = as.matrix(test_labels)
train_x=as.matrix(train_x)
test_x=as.matrix(test_x)

In [181]:
head(test_x)

Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
32,0,38.5,52.50,7.7,22.1,7.5,6.93,3.230,106.0,12.1,69.00
32,0,39.2,74.10,32.6,24.8,9.6,9.15,4.320,76.0,29.9,68.70
32,0,41.6,43.30,18.5,19.7,12.3,9.92,6.050,111.0,91.0,74.00
32,0,46.3,41.30,17.5,17.8,8.5,7.01,4.790,70.0,16.9,74.50
32,0,50.9,65.50,23.2,21.2,6.9,8.69,4.100,83.0,13.7,71.30
32,0,42.4,86.30,20.3,20.0,35.2,5.46,4.450,81.0,15.9,69.90


In [182]:
#Export to csvs
write.csv(train_labels,"/Users/shreya/Desktop/STAT 503/train_labels.csv", row.names = FALSE)
write.csv(test_labels,"/Users/shreya/Desktop/STAT 503/test_labels.csv", row.names = FALSE)
write.csv(train_x,"/Users/shreya/Desktop/STAT 503/train_x.csv", row.names = FALSE)
write.csv(test_x,"/Users/shreya/Desktop/STAT 503/test_x.csv", row.names = FALSE)